In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [3]:
basedir = './국토교통부_건축물대장_총괄표제부/'
filenames = [f for f in os.listdir(basedir) if f.endswith('.txt')]
filenames

['MART_DJY_02_201412.txt',
 'mart_djy_02_201512.txt',
 'mart_djy_02_201612.txt',
 'mart_djy_02_201712.txt',
 'mart_djy_02_201812.txt',
 'mart_djy_02_201912.txt',
 'mart_djy_02_202011.txt']

In [4]:
for file in filenames:
    f = open(basedir + file, 'r')
    print(f.readline())
    f.close()

관리_건축물대장_PK|대장_구분_코드|대장_구분_코드_명|대장_종류_코드|대장_종류_코드_명|신_구_대장_구분_코드|신_구_대장_구분_코드_명|대지_위치|도로명_대지_위치|건물_명|시군구_코드|법정동_코드|대지_구분_코드|번|지|특수지_명|블록|로트|외필지_수|새주소_도로_코드|새주소_법정동_코드|새주소_지상지하_코드|새주소_본_번|새주소_부_번|양성화_여부|특이사항|대지_면적(㎡)|건축_면적(㎡)|건폐율(%)|연면적(㎡)|용적률_산정_연면적(㎡)|용적률(%)|주_용도_코드|주_용도_코드_명|기타_용도|세대_수(세대)|가구_수(가구)|주_건축물_수|부속_건축물_수|부속_건축물_면적(㎡)|총_주차_수|옥내_기계식_대수(대)|옥내_기계식_면적(㎡)|옥외_기계식_대수(대)|옥외_기계식_면적(㎡)|옥내_자주식_대수(대)|옥내_자주식_면적(㎡)|옥외_자주식_대수(대)|옥외_자주식_면적(㎡)|허가_일|착공_일|사용승인_일|허가번호_년|허가번호_기관_코드|허가번호_기관_코드_명|허가번호_구분_코드|허가번호_구분_코드_명|호_수(호)|에너지_효율등급|에너지_절감율|EPI_점수|친환경_건축물_등급|친환경_건축물_인증점수|지능형_건축물_등급|지능형_건축물_인증점수|생성_일자

36110-100197529|1|일반|1|총괄표제부|1|신대장|세종특별자치시  금남면 도암리 233-3번지|||36110|34039|0|0233|0003||||1|||0|||478.4|129.67|27.1|129.67|129.67|27.1|01000|단독주택|제2종근린생활시설|0|1|2|0|0|0|0|0|0|0|0|0|0|0|20110822|20110824|20111104|2011|4560076|도시건축과|1202|증축신고|0||0|0||0||0|20140805

11500-100208246|1|일반|1|총괄표제부|1|신대장|서울특별시 강서구 외발산동 216-7번지|서울특별시 강서구 남부순환로17길 6||11500|10700|0|0216|0007||||0|115004145281|10702|0|6|0|409

In [5]:
colnames_df = pd.read_excel(basedir + '데이터구조.xls', header=1)
print(colnames_df.shape)
colnames_df.head()

(64, 3)


,컬럼 한글명,데이터 타입,비고
0,관리_건축물대장_PK,VARCHAR(33),NaN
1,대장_구분_코드,VARCHAR(1),NaN
2,대장_구분_코드_명,VARCHAR(100),NaN
3,대장_종류_코드,VARCHAR(1),NaN
4,대장_종류_코드_명,VARCHAR(100),NaN


In [8]:
colnames = colnames_df['컬럼 한글명'].tolist()
print(len(colnames))
colnames[:5]

64


['관리_건축물대장_PK', '대장_구분_코드', '대장_구분_코드_명', '대장_종류_코드', '대장_종류_코드_명']

In [11]:
dfs_list = []
for i, file in enumerate(filenames):
    if i == 0:
        df = pd.read_csv(basedir + file, encoding='euc-kr', sep='|')
        colnames_2014 = df.columns.tolist()
        colnames_2014.remove('양성화_여부')
        colnames_2014.remove('특이사항')
        colnames_2014.append('양성화_여부')
        colnames_2014.append('특이사항')
        df.columns = colnames_2014
        df.drop(columns=['양성화_여부', '특이사항'], inplace=True)
        
    else:
        df = pd.read_csv(basedir + file, encoding='euc-kr', sep='|', header=None, names=colnames)
    
    dfs_list.append(df)

c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (16,17,30,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (16,17,30,47,48,49,50,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (16,17,47,48,49,50,51,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (15,16,17,47,48,49,50,51,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactiv

In [12]:
for df in dfs_list:
    print(df.shape)

(535169, 64)
(546192, 64)
(554892, 64)
(563589, 64)
(572676, 64)
(581894, 64)
(588611, 64)


In [16]:
first_cols = dfs_list[0].columns

In [19]:
for df in dfs_list:
    for i in range(len(first_cols)):
        if first_cols[i] != df.columns[i]:
            print(first_cols[i])
            print(df.columns[i])

건폐율(%)
건폐_율(%)
용적률_산정_연면적(㎡)
용적_률_산정_연면적(㎡)
용적률(%)
용적_률(%)
에너지_효율등급
에너지효율_등급
에너지_절감율
에너지절감_율
EPI_점수
에너지_EPI점수
건폐율(%)
건폐_율(%)
용적률_산정_연면적(㎡)
용적_률_산정_연면적(㎡)
용적률(%)
용적_률(%)
에너지_효율등급
에너지효율_등급
에너지_절감율
에너지절감_율
EPI_점수
에너지_EPI점수
건폐율(%)
건폐_율(%)
용적률_산정_연면적(㎡)
용적_률_산정_연면적(㎡)
용적률(%)
용적_률(%)
에너지_효율등급
에너지효율_등급
에너지_절감율
에너지절감_율
EPI_점수
에너지_EPI점수
건폐율(%)
건폐_율(%)
용적률_산정_연면적(㎡)
용적_률_산정_연면적(㎡)
용적률(%)
용적_률(%)
에너지_효율등급
에너지효율_등급
에너지_절감율
에너지절감_율
EPI_점수
에너지_EPI점수
건폐율(%)
건폐_율(%)
용적률_산정_연면적(㎡)
용적_률_산정_연면적(㎡)
용적률(%)
용적_률(%)
에너지_효율등급
에너지효율_등급
에너지_절감율
에너지절감_율
EPI_점수
에너지_EPI점수
건폐율(%)
건폐_율(%)
용적률_산정_연면적(㎡)
용적_률_산정_연면적(㎡)
용적률(%)
용적_률(%)
에너지_효율등급
에너지효율_등급
에너지_절감율
에너지절감_율
EPI_점수
에너지_EPI점수


In [20]:
second_cols = dfs_list[1].columns

In [21]:
dfs_list[0].columns = second_cols

In [22]:
dfs_list[0].columns

Index(['관리_건축물대장_PK', '대장_구분_코드', '대장_구분_코드_명', '대장_종류_코드', '대장_종류_코드_명',
       '신_구_대장_구분_코드', '신_구_대장_구분_코드_명', '대지_위치', '도로명_대지_위치', '건물_명',
       '시군구_코드', '법정동_코드', '대지_구분_코드', '번', '지', '특수지_명', '블록', '로트', '외필지_수',
       '새주소_도로_코드', '새주소_법정동_코드', '새주소_지상지하_코드', '새주소_본_번', '새주소_부_번',
       '대지_면적(㎡)', '건축_면적(㎡)', '건폐_율(%)', '연면적(㎡)', '용적_률_산정_연면적(㎡)',
       '용적_률(%)', '주_용도_코드', '주_용도_코드_명', '기타_용도', '세대_수(세대)', '가구_수(가구)',
       '주_건축물_수', '부속_건축물_수', '부속_건축물_면적(㎡)', '총_주차_수', '옥내_기계식_대수(대)',
       '옥내_기계식_면적(㎡)', '옥외_기계식_대수(대)', '옥외_기계식_면적(㎡)', '옥내_자주식_대수(대)',
       '옥내_자주식_면적(㎡)', '옥외_자주식_대수(대)', '옥외_자주식_면적(㎡)', '허가_일', '착공_일',
       '사용승인_일', '허가번호_년', '허가번호_기관_코드', '허가번호_기관_코드_명', '허가번호_구분_코드',
       '허가번호_구분_코드_명', '호_수(호)', '에너지효율_등급', '에너지절감_율', '에너지_EPI점수',
       '친환경_건축물_등급', '친환경_건축물_인증점수', '지능형_건축물_등급', '지능형_건축물_인증점수', '생성_일자'],
      dtype='object')

In [23]:
first_cols = dfs_list[0].columns

In [24]:
for df in dfs_list:
    print(first_cols == df.columns)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True

In [25]:
concat_df = pd.concat(dfs_list)
print(concat_df.shape)
concat_df.head()

(3943023, 64)


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,신_구_대장_구분_코드,신_구_대장_구분_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),주_용도_코드,주_용도_코드_명,기타_용도,세대_수(세대),가구_수(가구),주_건축물_수,부속_건축물_수,부속_건축물_면적(㎡),총_주차_수,옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자
0,29110-10018306602,2,집합,1,총괄표제부,1.0,신대장,광주광역시 동구 지산동 671번지,NaN,지산휴먼시아,29110,11100,0,671,0.0,NaN,NaN,NaN,0,2.911043e+11,11102.0,0.0,20.0,0.0,21866.4,3959.29,18.11,48700.17,41857.51,191.42,02000,공동주택,"공동주택(아파트), 근린생활시설",375,0,7,9,6716.33,385,0,0.0,0,0.0,214,5575.94,171,2026.5,2.00507e+07,20080215,2.01006e+07,NaN,NaN,,NaN,,4,NaN,0.0,0,NaN,0,NaN,0,20100622
1,45111-178002,2,집합,1,총괄표제부,0.0,구대장,전라북도 전주시 완산구 평화동1가 570-15번지,NaN,극동평화아파트,45111,13200,0,570,15.0,NaN,NaN,NaN,0,NaN,NaN,0.0,NaN,NaN,0.0,0.00,0.00,16372.48,0.00,0.00,02000,공동주택,공동주택,0,0,0,0,0.00,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,NaN,NaN,NaN,,NaN,,0,NaN,0.0,0,NaN,0,NaN,0,20100619
2,45130-10019206502,1,일반,1,총괄표제부,1.0,신대장,전라북도 군산시 서수면 마룡리 93-16번지,NaN,NaN,45130,34024,0,93,16.0,NaN,NaN,NaN,0,4.513046e+11,34001.0,0.0,66.0,0.0,11550.0,4880.55,42.25,5025.80,5025.80,43.51,17000,공장,공장,0,0,3,0,0.00,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,NaN,1991,NaN,,NaN,,0,NaN,0.0,0,NaN,0,NaN,0,20100619
3,45130-10019323902,1,일반,1,총괄표제부,1.0,신대장,전라북도 군산시 오식도동 882번지,NaN,(주)대진테크닉,45130,14700,0,882,0.0,NaN,NaN,NaN,0,4.513033e+11,14701.0,0.0,46.0,0.0,9974.9,4623.22,46.35,4877.28,4877.28,48.90,17000,공장,공장,0,0,2,0,0.00,17,0,0.0,0,0.0,0,0.00,17,195.5,2.01002e+07,20100227,2.01006e+07,2010,6.45108e+06,개발지원부,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20100619
4,28110-10017349502,2,집합,1,총괄표제부,1.0,신대장,인천광역시 중구 을왕동 878-2번지,NaN,썬앤문하우스,28110,14900,0,878,2.0,NaN,NaN,NaN,3,2.811042e+11,14901.0,0.0,25.0,0.0,1829.0,365.20,19.97,1307.68,1307.68,71.50,02000,공동주택,다세대주택,16,0,2,0,0.00,16,0,0.0,0,0.0,0,0.00,16,189.0,2.00508e+07,20070801,2.00807e+07,NaN,NaN,,NaN,,0,NaN,0.0,0,NaN,0,NaN,0,20090317


In [29]:
def correct_addr(x):
    # x is a string
    if pd.isna(x) == True:
        return x
    else:
        x = x.replace('번지', '')
        if x[0] == ' ':
            for i in range(len(x)):
                if x[i] != ' ':
                    x = x[i:]
                    break
        
        splitted = x.split('-')
        if len(splitted) == 1:
            return x+'-0'
        else:
            return x

In [30]:
concat_df['지번주소'] = concat_df['대지_위치'].apply(correct_addr)
concat_df['도로명주소'] = concat_df['도로명_대지_위치'].apply(correct_addr)

In [31]:
concat_df['지번주소'].unique()[:10]

array(['광주광역시 동구 지산동 671-0', '전라북도 전주시 완산구 평화동1가 570-15',
       '전라북도 군산시 서수면 마룡리 93-16', '전라북도 군산시 오식도동 882-0',
       '인천광역시 중구 을왕동 878-2', '전라북도 익산시 오산면 신지리 21-26',
       '전라북도 정읍시 감곡면 통석리 826-0', '울산광역시 중구 다운동 910-0',
       '경기도 파주시 조리읍 장곡리 127-0', '경기도 화성시 양감면 송산리 320-0'], dtype=object)

In [32]:
concat_df['도로명주소'].unique()[:10]

array([nan, '서울특별시 강서구 남부순환로17길 6-0', '인천광역시 중구 왕산로52번길 25-0',
       '인천광역시 중구 선녀바위로 35-0', '서울특별시 강서구 화곡로66길 80-0',
       '서울특별시 강서구 양천로24가길 59-0', '서울특별시 강서구 양천로14길 118-0',
       '서울특별시 금천구 벚꽃로24길 24-0', '부산광역시 북구 기찰로 176-0',
       '부산광역시 기장군 한들길 21-0'], dtype=object)

In [33]:
concat_df.drop(columns=[col for col in concat_df.columns if col.endswith('_코드')], inplace=True)

In [34]:
concat_df.columns = [col.replace('_코드_명', '') for col in concat_df.columns]
print(concat_df.shape)
concat_df.head()

(3943023, 54)


,관리_건축물대장_PK,대장_구분,대장_종류,신_구_대장_구분,대지_위치,도로명_대지_위치,건물_명,번,지,특수지_명,블록,로트,외필지_수,새주소_본_번,새주소_부_번,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),주_용도,기타_용도,세대_수(세대),가구_수(가구),주_건축물_수,부속_건축물_수,부속_건축물_면적(㎡),총_주차_수,옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관,허가번호_구분,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,지번주소,도로명주소
0,29110-10018306602,집합,총괄표제부,신대장,광주광역시 동구 지산동 671번지,NaN,지산휴먼시아,671,0.0,NaN,NaN,NaN,0,20.0,0.0,21866.4,3959.29,18.11,48700.17,41857.51,191.42,공동주택,"공동주택(아파트), 근린생활시설",375,0,7,9,6716.33,385,0,0.0,0,0.0,214,5575.94,171,2026.5,2.00507e+07,20080215,2.01006e+07,NaN,,,4,NaN,0.0,0,NaN,0,NaN,0,20100622,광주광역시 동구 지산동 671-0,NaN
1,45111-178002,집합,총괄표제부,구대장,전라북도 전주시 완산구 평화동1가 570-15번지,NaN,극동평화아파트,570,15.0,NaN,NaN,NaN,0,NaN,NaN,0.0,0.00,0.00,16372.48,0.00,0.00,공동주택,공동주택,0,0,0,0,0.00,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,NaN,NaN,,,0,NaN,0.0,0,NaN,0,NaN,0,20100619,전라북도 전주시 완산구 평화동1가 570-15,NaN
2,45130-10019206502,일반,총괄표제부,신대장,전라북도 군산시 서수면 마룡리 93-16번지,NaN,NaN,93,16.0,NaN,NaN,NaN,0,66.0,0.0,11550.0,4880.55,42.25,5025.80,5025.80,43.51,공장,공장,0,0,3,0,0.00,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,NaN,1991,,,0,NaN,0.0,0,NaN,0,NaN,0,20100619,전라북도 군산시 서수면 마룡리 93-16,NaN
3,45130-10019323902,일반,총괄표제부,신대장,전라북도 군산시 오식도동 882번지,NaN,(주)대진테크닉,882,0.0,NaN,NaN,NaN,0,46.0,0.0,9974.9,4623.22,46.35,4877.28,4877.28,48.90,공장,공장,0,0,2,0,0.00,17,0,0.0,0,0.0,0,0.00,17,195.5,2.01002e+07,20100227,2.01006e+07,2010,개발지원부,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20100619,전라북도 군산시 오식도동 882-0,NaN
4,28110-10017349502,집합,총괄표제부,신대장,인천광역시 중구 을왕동 878-2번지,NaN,썬앤문하우스,878,2.0,NaN,NaN,NaN,3,25.0,0.0,1829.0,365.20,19.97,1307.68,1307.68,71.50,공동주택,다세대주택,16,0,2,0,0.00,16,0,0.0,0,0.0,0,0.00,16,189.0,2.00508e+07,20070801,2.00807e+07,NaN,,,0,NaN,0.0,0,NaN,0,NaN,0,20090317,인천광역시 중구 을왕동 878-2,NaN


In [35]:
concat_df.drop(columns=['대지_위치', '도로명_대지_위치', '건물_명', '번', '지', '특수지_명', '블록', '로트', '새주소_본_번',
                        '새주소_부_번', '허가번호_기관', '허가번호_구분'
                       ], inplace=True)

In [36]:
concat_df['총세대수'] = concat_df['세대_수(세대)'] + concat_df['가구_수(가구)'] + concat_df['호_수(호)']
concat_df.drop(columns=['세대_수(세대)', '가구_수(가구)', '호_수(호)'], inplace=True)
print(concat_df.shape)
concat_df.head()

(3943023, 40)


,관리_건축물대장_PK,대장_구분,대장_종류,신_구_대장_구분,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),주_용도,기타_용도,주_건축물_수,부속_건축물_수,부속_건축물_면적(㎡),총_주차_수,옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,지번주소,도로명주소,총세대수
0,29110-10018306602,집합,총괄표제부,신대장,0,21866.4,3959.29,18.11,48700.17,41857.51,191.42,공동주택,"공동주택(아파트), 근린생활시설",7,9,6716.33,385,0,0.0,0,0.0,214,5575.94,171,2026.5,2.00507e+07,20080215,2.01006e+07,NaN,NaN,0.0,0,NaN,0,NaN,0,20100622,광주광역시 동구 지산동 671-0,NaN,379
1,45111-178002,집합,총괄표제부,구대장,0,0.0,0.00,0.00,16372.48,0.00,0.00,공동주택,공동주택,0,0,0.00,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,0,NaN,0,20100619,전라북도 전주시 완산구 평화동1가 570-15,NaN,0
2,45130-10019206502,일반,총괄표제부,신대장,0,11550.0,4880.55,42.25,5025.80,5025.80,43.51,공장,공장,3,0,0.00,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,NaN,1991,NaN,0.0,0,NaN,0,NaN,0,20100619,전라북도 군산시 서수면 마룡리 93-16,NaN,0
3,45130-10019323902,일반,총괄표제부,신대장,0,9974.9,4623.22,46.35,4877.28,4877.28,48.90,공장,공장,2,0,0.00,17,0,0.0,0,0.0,0,0.00,17,195.5,2.01002e+07,20100227,2.01006e+07,2010,NaN,0.0,0,NaN,0,NaN,0,20100619,전라북도 군산시 오식도동 882-0,NaN,0
4,28110-10017349502,집합,총괄표제부,신대장,3,1829.0,365.20,19.97,1307.68,1307.68,71.50,공동주택,다세대주택,2,0,0.00,16,0,0.0,0,0.0,0,0.00,16,189.0,2.00508e+07,20070801,2.00807e+07,NaN,NaN,0.0,0,NaN,0,NaN,0,20090317,인천광역시 중구 을왕동 878-2,NaN,16


In [37]:
concat_df.drop(columns=['생성_일자'], inplace=True)

In [38]:
print(concat_df.shape)
concat_df.drop_duplicates(inplace=True)
concat_df.shape

(3943023, 39)


(2040596, 39)

In [39]:
seoul_df = concat_df[concat_df['지번주소'].str.contains('서울')]
print(seoul_df.shape)
seoul_df.head()

(63577, 39)


,관리_건축물대장_PK,대장_구분,대장_종류,신_구_대장_구분,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),주_용도,기타_용도,주_건축물_수,부속_건축물_수,부속_건축물_면적(㎡),총_주차_수,옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,지번주소,도로명주소,총세대수
267,11650-20602,집합,총괄표제부,구대장,0,359.3,162.78,45.30,493.41,325.56,90.61,공동주택,공동주택,2,0,0.00,2,0,0.0,0,0.0,0,0.00,2,30.0,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,0,NaN,0,서울특별시 서초구 방배동 911-29,NaN,8
268,11650-21702,집합,총괄표제부,신대장,0,4321.4,1855.67,42.94,12660.24,6129.15,141.83,공동주택,공동주택(연립주택),3,0,0.00,0,0,0.0,0,0.0,110,4522.93,0,0.0,1.99909e+07,19991025,2.00304e+07,NaN,NaN,0.0,0,NaN,0,NaN,0,서울특별시 서초구 서초동 1496-26,NaN,18
269,11650-25302,일반,총괄표제부,신대장,0,0.0,0.00,0.00,239.73,0.00,0.00,단독주택,"주택, 점포",3,0,0.00,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,0,NaN,0,서울특별시 서초구 방배동 938-24,NaN,1
270,11650-29902,일반,총괄표제부,구대장,0,227.9,106.13,46.57,121.73,121.73,53.41,창고시설,"창고시설, 제2종근린생활시설",2,0,0.00,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,0,NaN,0,서울특별시 서초구 잠원동 32-9,NaN,0
271,11650-32802,일반,총괄표제부,신대장,0,0.0,0.00,0.00,964.68,0.00,0.00,문화및집회시설,종교시설,3,1,17.26,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,0,NaN,0,서울특별시 서초구 반포동 709-5,NaN,2


In [40]:
seoul_df.to_csv('./국토교통부_건축물대장_총괄표제부/wbdinfo_checkpoint_20210116_1304.csv', index=False)